In [1]:
import julia 
from julia import Julia
jl = Julia(compiled_modules=False)  # Initialize Julia interpreter
Julia(runtime="/home/jxxiong/julia-1.9.2/bin/julia")
from julia import Base
# Now you can call functions from Julia's Base module
from julia import Main

In [1]:
import numpy as np
import gurobipy as gp

from gurobipy import Model, GRB, QuadExpr, LinExpr
from scipy.sparse import csc_matrix, csr_matrix
import pickle

In [2]:
# def get_constraints(m):
#     ini_ineq_matrix = []
#     ini_ineq_rhs = []
#     ini_eq_matrix = []
#     ini_eq_rhs = []
#     a_matrix = m.getA().toarray()

#     i = 0
#     for constr in m.getConstrs():
#         if constr.sense == '<':
#             ini_ineq_matrix.append(a_matrix[i,:])
#             ini_ineq_rhs.append(constr.RHS)
#             i += 1
#         elif constr.sense == '>':
#             ini_ineq_matrix.append(-a_matrix[i,:])
#             ini_ineq_rhs.append(-constr.RHS)
#             i += 1
#         elif constr.sense == '=':
#             ini_eq_matrix.append(a_matrix[i,:])
#             ini_eq_rhs.append(constr.RHS)
#             i += 1

#     ini_ineq_matrix = np.array(ini_ineq_matrix)
#     ini_ineq_rhs = np.array(ini_ineq_rhs)
#     ini_eq_matrix = np.array(ini_eq_matrix)
#     ini_eq_rhs = np.array(ini_eq_rhs)

#     return ini_ineq_matrix, ini_ineq_rhs, ini_eq_matrix, ini_eq_rhs

# def get_objective(m):
#     ini_q_matrix = m.getQ().toarray()
#     ini_p_vec = []
#     variables = m.getVars()
#     for var in variables:
#         ini_p_vec.append(var.obj)
#     ini_p_vec = np.array(ini_p_vec)

#     return ini_q_matrix, ini_p_vec

# def get_bound(m):
#     variables = m.getVars()
#     ini_lb = []
#     ini_ub = []
#     for var in variables:
#         ini_lb.append(var.lb)
#         ini_ub.append(var.ub)
#     ini_lb = np.array(ini_lb)
#     ini_ub = np.array(ini_ub)

#     return ini_lb, ini_ub

# def perturb_eq_rhs(ini_eq_rhs, num_samples, perturb_rate=perturb_rate):
#     eq_rhs_list = []
#     for _ in range(num_samples):
#         eq_rhs = ini_eq_rhs.copy()
#         nonzero_indices = np.nonzero(ini_eq_rhs)
#         nonzero_values = ini_eq_rhs[nonzero_indices]
#         perturbed_values = nonzero_values * (1 + np.random.uniform(-perturb_rate, perturb_rate, size=nonzero_values.shape))
#         eq_rhs[nonzero_indices] = perturbed_values
#         eq_rhs_list.append(eq_rhs)
#     return np.array(eq_rhs_list)

In [3]:
m = gp.read('/home/jxxiong/A-xjx/DC3/datasets/acopf/model_3.mps')
var_index = {v.VarName: i for i, v in enumerate(m.getVars())}
var_index

Read MPS format model from file /home/jxxiong/A-xjx/DC3/datasets/acopf/model_3.mps
Reading time = 0.00 seconds
: 7 rows, 24 columns, 19 nonzeros


{'0_vr[2]': 0,
 '0_vr[3]': 1,
 '0_vr[1]': 2,
 '0_vi[2]': 3,
 '0_vi[3]': 4,
 '0_vi[1]': 5,
 '0_pg[2]': 6,
 '0_pg[3]': 7,
 '0_pg[1]': 8,
 '0_qg[2]': 9,
 '0_qg[3]': 10,
 '0_qg[1]': 11,
 '0_p[(2,_3,_2)]': 12,
 '0_p[(3,_1,_2)]': 13,
 '0_p[(1,_1,_3)]': 14,
 '0_p[(2,_2,_3)]': 15,
 '0_p[(3,_2,_1)]': 16,
 '0_p[(1,_3,_1)]': 17,
 '0_q[(2,_3,_2)]': 18,
 '0_q[(3,_1,_2)]': 19,
 '0_q[(1,_1,_3)]': 20,
 '0_q[(2,_2,_3)]': 21,
 '0_q[(3,_2,_1)]': 22,
 '0_q[(1,_3,_1)]': 23}

In [4]:
Q = m.getQ().toarray()
p = []
variables = m.getVars()
for var in variables:
    p.append(var.obj)
p = np.array(p)
Q.shape, p.shape

((24, 24), (24,))

In [5]:
Lb = np.array([v.lb for v in m.getVars()])
Ub = np.array([v.ub for v in m.getVars()])
print(Lb.shape, Ub.shape)
print(Lb.min(), Lb.max())
print(Ub.min(), Ub.max())

(24,) (24,)
-90.0 0.0
0.0 90.0


In [6]:
a_matrix = m.getA().toarray()
print(a_matrix.shape)

num_var = len(m.getVars())
print(num_var)

(7, 24)
24


In [7]:
eq_lin_constrs = [c for c in m.getConstrs() if c.sense == '='] 
eq_quad_constrs = [qc for qc in m.getQConstrs() if qc.qcsense == '=']
ineq_lin_constrs = [c for c in m.getConstrs() if c.sense != '=']
ineq_quad_constrs = [qc for qc in m.getQConstrs() if qc.qcsense != '=']
print(len(eq_lin_constrs), len(eq_quad_constrs), len(eq_lin_constrs)+len(eq_quad_constrs))
print(len(ineq_lin_constrs), len(ineq_quad_constrs), len(ineq_lin_constrs)+len(ineq_quad_constrs))

7 12 19
0 18 18


## get the coefficients for equality linear constraints

In [8]:
eq_lin_matrix = []
eq_lin_rhs = []
a_matrix = m.getA().toarray()
for i, constr in enumerate(eq_lin_constrs):
    if constr.sense == '=':
        eq_lin_matrix.append(a_matrix[i,:])
        eq_lin_rhs.append(constr.RHS)
eq_lin_matrix = np.array(eq_lin_matrix)
eq_lin_rhs = np.array(eq_lin_rhs)
print(eq_lin_matrix.shape, eq_lin_rhs.shape)

(7, 24) (7,)


In [9]:
eq_lin_rhs

array([ 0.  , -1.1 , -0.4 , -0.95, -0.5 , -1.1 , -0.4 ])

## get the coefficients for equality quadratic constraints

$y'Hy + g'y  \leq h$

In [10]:
eq_quad_H = []
eq_quad_g = []
eq_quad_h = []

for qc in eq_quad_constrs:
    H_row = []
    H_col = []
    H_data = []
    
    g_tmp = np.zeros(num_var)
    if qc.qcsense == '=':
        expr = m.getQCRow(qc)
        print(expr)
        # Quadratic terms
        for i in range(expr.size()):
            var1 = expr.getVar1(i)
            var2 = expr.getVar2(i)
            coeff = expr.getCoeff(i)
            idx1 = var_index[var1.VarName]
            idx2 = var_index[var2.VarName]
            H_row.append(idx1)
            H_col.append(idx2)
            H_data.append(coeff)
        print(H_row)
        print(H_col)
        print(H_data)
        # Linear part of the quadratic constraint
        lin_expr = expr.getLinExpr()
        eq_quad_h.append(qc.getAttr('qcrhs'))
        for j in range(lin_expr.size()):
            var = lin_expr.getVar(j)
            coeff = lin_expr.getCoeff(j)
            g_tmp[var_index[var.VarName]] = coeff
        print(qc.getAttr('qcrhs'))
        print(g_tmp)

        H_tmp = csc_matrix((np.array(H_data), (np.array(H_row), np.array(H_col))), shape=(num_var, num_var))
        eq_quad_H.append(H_tmp)
        eq_quad_g.append(g_tmp)

eq_quad_H = np.array(eq_quad_H)
eq_quad_g = np.array(eq_quad_g)
eq_quad_h = np.array(eq_quad_h)

0_p[(2,_3,_2)] + [ 0.08879023307436183 0_vr[2] * 0_vr[3] + -2.6637069922308547 0_vr[2] * 0_vi[3] + -0.08879023307436183 0_vr[3] ^ 2 + 2.6637069922308547 0_vr[3] * 0_vi[2] + 0.08879023307436183 0_vi[2] * 0_vi[3] + -0.08879023307436183 0_vi[3] ^ 2 ]
[0, 0, 1, 1, 3, 4]
[1, 4, 1, 3, 4, 4]
[0.08879023307436183, -2.6637069922308547, -0.08879023307436183, 2.6637069922308547, 0.08879023307436183, -0.08879023307436183]
0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0_q[(2,_3,_2)] + [ 2.6637069922308547 0_vr[2] * 0_vr[3] + 0.08879023307436183 0_vr[2] * 0_vi[3] + -1.9637069922308548 0_vr[3] ^ 2 + -0.08879023307436183 0_vr[3] * 0_vi[2] + 2.6637069922308547 0_vi[2] * 0_vi[3] + -1.9637069922308548 0_vi[3] ^ 2 ]
[0, 0, 1, 1, 3, 4]
[1, 4, 1, 3, 4, 4]
[2.6637069922308547, 0.08879023307436183, -1.9637069922308548, -0.08879023307436183, 2.6637069922308547, -1.9637069922308548]
0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
0_p[(2,_2,_3)] + [ -

In [11]:
import cyipopt as ipopt

class nonconvex_ipopt(object):
    def __init__(self, Q, p, A, eq_H, eq_g, ineq_H=None, ineq_g=None):
        self.Q = Q
        self.p = p
        self.A = A
        self.eq_H = eq_H
        self.eq_g = eq_g
        self.ineq_H = ineq_H
        self.ineq_g = ineq_g
        # self.tril_indices = np.tril_indices(Q.shape[0])

    def objective(self, y):
        return (y.T @ self.Q @ y) + self.p.T@y

    def gradient(self, y):
        return 2*self.Q@y + self.p

    def constraints(self, y):
        constr = [self.A@y]
        for i in range(len(self.eq_H)):
            # print(self.H[i].shape)
            constr.append(y.T @ self.eq_H[i] @ y + self.eq_g[i].T@y)
        if self.ineq_H is not None:
            for i in range(len(self.ineq_H)):
                constr.append(y.T @ self.ineq_H[i] @ y + self.ineq_g[i].T@y)
        return np.hstack(constr)

    def jacobian(self, y):
        jac = [self.A.flatten()]
        for i in range(len(self.eq_H)):
            jac.append((2*self.eq_H[i]@y + self.eq_g[i]).flatten())
        
        if self.ineq_H is not None:
            for i in range(len(self.ineq_H)):
                jac.append((2*self.ineq_H[i]@y + self.ineq_g[i]).flatten())
        return np.concatenate(jac)


def build_ipopt(Q, p, A, Xi, eq_H, eq_g, eq_h, ineq_H=None, ineq_g=None, ineq_h=None, lb=None, ub=None, y0=None, solver_type='ipopt', tol=1e-4):
    num_var = A.shape[1]
    # upper and lower bounds on variables
    if lb is None:
        lb = -np.infty * np.ones(num_var)
    if ub is None:
        ub = np.infty * np.ones(num_var)

    # upper and lower bounds on constraints
    cl = [Xi, eq_h]
    cu = [Xi, eq_h]
    
    if ineq_h is not None:
        cl.append(-np.infty * np.ones(len(ineq_h)))
        cu.append(ineq_h)
    
    cl = np.hstack(cl)
    cu = np.hstack(cu)

    nlp = ipopt.problem(n=num_var,
                        m=len(cl),
                        problem_obj=nonconvex_ipopt(Q, p, A, eq_H, eq_g, ineq_H, ineq_g),
                        lb=lb,
                        ub=ub,
                        cl=cl,
                        cu=cu)

    nlp.add_option('tol', tol)
    nlp.add_option('print_level', 0) # 3)

    # start_time = time.time()
    # y, info = nlp.solve(y0)
    # end_time = time.time()
    return nlp

In [12]:
m_ipopt = build_ipopt(Q, p, eq_lin_matrix, eq_lin_rhs, eq_quad_H, eq_quad_g, eq_quad_h, lb=Lb, ub=Ub)

/tmp/ipykernel_2971757/678030480.py:60: FutureWarning: The class named 'problem' will soon be deprecated in CyIpopt. Please replace all uses and use 'Problem' going forward.
  nlp = ipopt.problem(n=num_var,


In [13]:
import time
start_time = time.time()
y, info = m_ipopt.solve(np.zeros(num_var))
print(time.time() - start_time)
print(info['obj_val'])


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

0.04996943473815918
3041.4999991935365


## get coefficient for inequality quad constraints

In [14]:
ineq_quad_H = []
ineq_quad_g = []
ineq_quad_h = []

for qc in ineq_quad_constrs:
    H_row = []
    H_col = []
    H_data = []
    
    g_tmp = np.zeros(num_var)
    if qc.qcsense == '<':
        expr = m.getQCRow(qc)
        # print(expr)
        # Quadratic terms
        for i in range(expr.size()):
            var1 = expr.getVar1(i)
            var2 = expr.getVar2(i)
            coeff = expr.getCoeff(i)
            idx1 = var_index[var1.VarName]
            idx2 = var_index[var2.VarName]
            H_row.append(idx1)
            H_col.append(idx2)
            H_data.append(coeff)

        # Linear part of the quadratic constraint
        lin_expr = expr.getLinExpr()
        ineq_quad_h.append(qc.getAttr('qcrhs'))
        for j in range(lin_expr.size()):
            var = lin_expr.getVar(j)
            coeff = lin_expr.getCoeff(j)
            # G[var_index[var.VarName]] += coeff
            g_tmp[var_index[var.VarName]] = coeff

        H_tmp = csc_matrix((np.array(H_data), (np.array(H_row), np.array(H_col))), shape=(num_var, num_var))
        ineq_quad_H.append(H_tmp)
        ineq_quad_g.append(g_tmp)
    elif qc.qcsense == '>':
        expr = m.getQCRow(qc)
        # print(expr)
        # Quadratic terms
        for i in range(expr.size()):
            var1 = expr.getVar1(i)
            var2 = expr.getVar2(i)
            coeff = expr.getCoeff(i)
            idx1 = var_index[var1.VarName]
            idx2 = var_index[var2.VarName]
            H_row.append(idx1)
            H_col.append(idx2)
            H_data.append(-coeff)
        # Linear part of the quadratic constraint
        lin_expr = expr.getLinExpr()
        ineq_quad_h.append(-qc.getAttr('qcrhs'))
        for j in range(lin_expr.size()):
            var = lin_expr.getVar(j)
            coeff = lin_expr.getCoeff(j)
            g_tmp[var_index[var.VarName]] = -coeff

        H_tmp = csc_matrix((np.array(H_data), (np.array(H_row), np.array(H_col))), shape=(num_var, num_var))
        ineq_quad_H.append(H_tmp)
        ineq_quad_g.append(g_tmp)
        
ineq_quad_H = np.array(ineq_quad_H)
ineq_quad_g = np.array(ineq_quad_g)
ineq_quad_h = np.array(ineq_quad_h)

In [15]:
ineq_quad_h

array([-8.10e-01, -8.10e-01, -8.10e-01,  0.00e+00,  2.50e-01,  2.50e-01,
        0.00e+00,  8.10e+03,  8.10e+03,  0.00e+00,  8.10e+03,  8.10e+03,
        1.21e+00,  1.21e+00,  1.21e+00, -0.00e+00, -0.00e+00, -0.00e+00])

In [17]:
m_ipopt_full = build_ipopt(Q, p, eq_lin_matrix, eq_lin_rhs, eq_quad_H, eq_quad_g, eq_quad_h, ineq_quad_H, ineq_quad_g, ineq_quad_h, Lb, Ub)
start_time = time.time()
y, info = m_ipopt_full.solve(np.zeros(num_var))
print(time.time() - start_time)
print(info['obj_val'])

0.047315120697021484
3041.5000000000036


/tmp/ipykernel_2971757/678030480.py:60: FutureWarning: The class named 'problem' will soon be deprecated in CyIpopt. Please replace all uses and use 'Problem' going forward.
  nlp = ipopt.problem(n=num_var,


In [28]:
model = Model('qcqp1')

# Create variables
x = model.addVars(num_var, lb=Lb, ub=Ub, name='x')

# Set objective
obj = QuadExpr()
for i in range(num_var):
    for j in range(num_var):
        obj.add(x[i]*Q[i,j]*x[j])

for i in range(num_var):
    obj.add(p[i]*x[i])
model.setObjective(obj)

# Add linear constraints
for i in range(len(eq_lin_matrix)):
    expr = LinExpr()
    for j in range(num_var):
        expr.add(x[j], eq_lin_matrix[i,j])
    model.addConstr(expr, GRB.EQUAL, eq_lin_rhs[i])

# Add quadratic constraints
for i in range(len(eq_quad_H)):
    expr = QuadExpr()
    for j in range(num_var):
        for k in range(num_var):
            expr.add(x[j]*eq_quad_H[i][j,k]*x[k])
    for j in range(num_var):
        expr.add(x[j]*eq_quad_g[i][j])
    expr.add(-eq_quad_h[i])
    model.addQConstr(expr, GRB.EQUAL, 0)

for i in range(len(ineq_quad_H)):
    expr = QuadExpr()
    for j in range(num_var):
        for k in range(num_var):
            expr.add(x[j]*ineq_quad_H[i][j,k]*x[k])
    for j in range(num_var):
        expr.add(x[j]*ineq_quad_g[i][j])
    expr.add(-ineq_quad_h[i])
    model.addQConstr(expr, GRB.LESS_EQUAL, 0)

/tmp/ipykernel_2971409/738468994.py:21: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  model.addConstr(expr, GRB.EQUAL, eq_lin_rhs[i])


In [35]:
m = gp.read('/home/jxxiong/A-xjx/DC3/datasets/acopf/model_3.mps')
for c in m.getConstrs():
    if c.Sense != '=':
        m.remove(c)

for qc in m.getQConstrs():
    if qc.qcsense != '=':
        m.remove(qc)

m.optimize()

Read MPS format model from file /home/jxxiong/A-xjx/DC3/datasets/acopf/model_3.mps
Reading time = 0.00 seconds
: 7 rows, 24 columns, 19 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 7 rows, 24 columns and 19 nonzeros
Model fingerprint: 0xbef4af69
Model has 2 quadratic objective terms
Model has 12 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [9e-02, 3e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+02, 5e+02]
  QObjective range [2e+03, 2e+03]
  Bounds range     [5e-01, 9e+01]
  RHS range        [4e-01, 1e+00]
Presolve removed 1 rows and 2 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 2 rows and 5 columns
Presolve time: 0.00s
Presolved: 84 rows, 36 columns, 261 non

# check

In [80]:
model = gp.Model("my_model")

# Create variables
x = model.addVar(name="x")
y = model.addVar(name="y")
model.update()

# Dictionary to map variable names to indices
var_index = {v.VarName: i for i, v in enumerate(model.getVars())}
var_index

{'x': 0, 'y': 1}

In [85]:
import gurobipy as gp
import numpy as np

# Create a new model
model = gp.Model("my_model")

# Create variables
x = model.addVar(name="x")
y = model.addVar(name="y")
model.update()

# Dictionary to map variable names to indices
var_index = {v.VarName: i for i, v in enumerate(model.getVars())}

# Add linear constraint
model.addConstr(2*x + 3*y == 7, name="linear_constr")

# Add quadratic constraint
model.addQConstr(x*x + 4*x*y + y*y + 2*x - 3*y <= 9, name="quad_constr")
model.addQConstr(0.5 * x*x + 2*x*y + y*y + 2*x - 3*y <= 9, name="quad_constr")

# Optimize the model
model.optimize()

# Initialize matrices for constraints
num_vars = model.numVars
A = np.zeros((model.numConstrs, num_vars))
b = np.zeros(model.numConstrs)
H = np.zeros((num_vars, num_vars))
G = np.zeros(num_vars)
h = 0

# Collect linear constraints
for i, constr in enumerate(model.getConstrs()):
    expr = model.getRow(constr)
    b[i] = constr.RHS
    for j in range(expr.size()):
        var = expr.getVar(j)
        coeff = expr.getCoeff(j)
        A[i, var_index[var.VarName]] = coeff

print("Matrix A:\n", A)
print("Vector b:\n", b)

# Collect quadratic constraints
for qc in model.getQConstrs():
    expr = model.getQCRow(qc)
    # Quadratic terms
    for i in range(expr.size()):
        var1 = expr.getVar1(i)
        var2 = expr.getVar2(i)
        coeff = expr.getCoeff(i)
        idx1 = var_index[var1.VarName]
        idx2 = var_index[var2.VarName]
        H[idx1, idx2] += coeff
        if idx1 != idx2:
            H[idx2, idx1] += coeff  # Ensure symmetry

    # Linear part of the quadratic constraint
    lin_expr = expr.getLinExpr()
    h -= lin_expr.getConstant()
    for j in range(lin_expr.size()):
        var = lin_expr.getVar(j)
        coeff = lin_expr.getCoeff(j)
        G[var_index[var.VarName]] += coeff

print("Matrix H:\n", H)
print("Vector G:\n", G)
print("Scalar h:\n", h)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0xe16390d1
Model has 2 quadratic constraints
Coefficient statistics:
  Matrix range     [2e+00, 3e+00]
  QMatrix range    [5e-01, 4e+00]
  QLMatrix range   [2e+00, 3e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 7e+00]
  QRHS range       [9e+00, 9e+00]

Continuous model is non-convex -- solving as a MIP

Presolve time: 0.00s
Presolved: 6 rows, 7 columns, 16 nonzeros
Presolved model has 3 quadratic constraint(s)
Presolved model has 1 bilinear constraint(s)
Variable types: 7 continuous, 0 integer (0 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work uni

In [62]:
qc = model.getQConstrs()[0]
expr = model.getQCRow(qc)

In [84]:
expr = model.getRow(constr)
expr

<gurobi.LinExpr: 2.0 x + 3.0 y>

In [68]:
expr.getVar1(0), expr.getVar2(0), expr.getCoeff(0)

(<gurobi.Var x (value 0.6242366733407322)>,
 <gurobi.Var x (value 0.6242366733407322)>,
 1.0)

In [69]:
expr.getVar1(1), expr.getVar2(1), expr.getCoeff(1)

(<gurobi.Var x (value 0.6242366733407322)>,
 <gurobi.Var y (value 1.9171755511061788)>,
 4.0)

In [70]:
expr.getVar1(2), expr.getVar2(2), expr.getCoeff(2)

(<gurobi.Var y (value 1.9171755511061788)>,
 <gurobi.Var y (value 1.9171755511061788)>,
 1.0)

In [73]:
lin_expr = expr.getLinExpr()
lin_expr.getConstant()

0.0

In [40]:
expr.getCoeff(0)
expr.getCoeff(3)

IndexError: list index out of range

In [36]:
expr

<gurobi.QuadExpr: 2.0 x + -3.0 y + [ x ^ 2 + 4.0 x * y + y ^ 2 ]>

<gurobi.Var y (value 1.9171755511061788)>


In [ ]:
import torch
from torch.autograd import Function
import numpy as np
import cvxpy as cp

class QCQPLayer(Function):
    @staticmethod
    def forward(ctx, z, Q, A, b, c, d):
        # Convert torch tensors to numpy for cvxpy compatibility
        z_np = z.detach().numpy()
        Q_np = Q.detach().numpy()
        A_np = A.detach().numpy()
        b_np = b.detach().numpy()
        c_np = c.detach().numpy()

        # Define cvxpy problem
        z_var = cp.Variable(len(z_np))
        objective = cp.Minimize(0.5 * cp.quad_form(z_var, Q_np) + c_np.T @ z_var)
        constraints = [0.5 * cp.quad_form(z_var, A_np) + b_np.T @ z_var + d == 0]
        prob = cp.Problem(objective, constraints)
        prob.solve()

        z_star = np.array(z_var.value).flatten()
        ctx.save_for_backward(z, Q, A, b, c, torch.tensor(z_star))

        return torch.tensor(z_star, dtype=torch.float32)

    @staticmethod
    def backward(ctx, grad_output):
        z, Q, A, b, c, z_star = ctx.saved_tensors
        # Sizes
        n = z.size(0)

        # Convert grad_output to numpy
        grad_output_np = grad_output.detach().numpy()

        # Define dual variables
        lambda_var = cp.Variable(1)

        # Differentiate KKT conditions w.r.t. z
        dz_dz = cp.Parameter(n, nonneg=True)

        # Stationarity condition from KKT
        stationarity = Q @ z_star + A.T @ lambda_var + c

        # Differentiating the stationarity and primal feasibility conditions
        stationarity_diff = cp.diff(stationarity, z_star)
        primal_feasibility_diff = cp.diff(A @ z_star + b, z_star)

        # Solve for gradients
        # Combine equations and solve as a linear system
        matrix = np.vstack([
            np.hstack([stationarity_diff, A.T]),
            np.hstack([A, np.zeros((1, 1))])
        ])
        vector = -np.hstack([stationarity.detach().numpy(), (A @ z_star + b).detach().numpy()])
        solutions = np.linalg.solve(matrix, vector)

        dz_star_dz = solutions[:n]
        dlambda_dz = solutions[n:]

        # Apply chain rule
        grad_input = dz_star_dz.T @ grad_output_np

        return torch.tensor(grad_input, dtype=torch.float32), None, None, None, None, None

# Example usage
Q = torch.tensor([[2.0, 0.0], [0.0, 2.0]], requires_grad=False)
A = torch.tensor([[1.0, 1.0]], requires_grad=False)
b = torch.tensor([-1.0], requires_grad=False)
c = torch.tensor([0.0, 0.0], requires_grad=False)
d = -0.5

z = torch.tensor([0.1, -0.2], requires_grad=True)
z_star = QCQPLayer.apply(z, Q, A, b, c, d)

loss = (z_star ** 2).sum()  # Dummy loss for demonstration
loss.backward()
print("z.grad:", z.grad)


In [4]:
ReaderFactory('mps')